The collaborative filter approach focuses on finding users who have given similar ratings to the same restaurant, thus creating a link between users, to whom will be suggested restaurants that were reviewed in a positive way. In this way, we look for associations between users, not between restaurants.

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [27]:
resto_rating = pd.read_csv('final_ratings.csv', sep=',', on_bad_lines="skip", encoding="latin-1")
#cols = ['rating', 'food_rating', 'service_rating']
#resto_rating.drop(cols, axis=1, inplace=True)
resto_rating.head()

,userID,restoID,rating
0,U1001,R0001,4
1,U1002,R0001,5
2,U1003,R0001,4
3,U1004,R0001,3
4,U1005,R0001,5


In [28]:
resto_rating.head(3)

,userID,restoID,rating
0,U1001,R0001,4
1,U1002,R0001,5
2,U1003,R0001,4


In [29]:
rating_count = (resto_rating.
     groupby(by = ['restoID'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_resto'})
     [['restoID', 'RatingCount_resto']]
    )
rating_count.head()

,restoID,RatingCount_resto
0,R0001,29
1,R0002,50
2,R0003,24
3,R0004,50
4,R0005,20


In [30]:
threshold = 0
rating_count = rating_count.query('RatingCount_resto >= @threshold')
rating_count.head()

,restoID,RatingCount_resto
0,R0001,29
1,R0002,50
2,R0003,24
3,R0004,50
4,R0005,20


In [31]:
resto_rating.head(3)

,userID,restoID,rating
0,U1001,R0001,4
1,U1002,R0001,5
2,U1003,R0001,4


In [32]:
user_rating = pd.merge(rating_count, resto_rating, left_on='restoID', right_on='restoID', how='left')

In [33]:
user_rating.head(3)

,restoID,RatingCount_resto,userID,rating
0,R0001,29,U1001,4
1,R0001,29,U1002,5
2,R0001,29,U1003,4


In [34]:
user_count = (user_rating.
     groupby(by = ['userID'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_user'})
     [['userID', 'RatingCount_user']]
    )
user_count.head()

,userID,RatingCount_user
0,U1001,15
1,U1002,15
2,U1003,12
3,U1004,12
4,U1005,12


In [35]:
threshold = 0
user_count = user_count.query('RatingCount_user >= @threshold')
user_count.head(200)

,userID,RatingCount_user
0,U1001,15
1,U1002,15
2,U1003,12
3,U1004,12
4,U1005,12
...,...,...
131,U1132,1
132,U1133,1
133,U1134,1
134,U1135,1


In [36]:
combined = user_rating.merge(user_count, left_on = 'userID', right_on = 'userID', how = 'inner')

In [37]:
combined.head(3)

,restoID,RatingCount_resto,userID,rating,RatingCount_user
0,R0001,29,U1001,4,15
1,R0002,50,U1001,5,15
2,R0003,24,U1001,5,15


In [38]:
combined.shape

(765, 5)

In [39]:
print('Number of unique resto: ', combined['restoID'].nunique()) #harusnya ada 18 resto
print('Number of unique users: ', combined['userID'].nunique()) #ini 136 harusnya user karena ada 2 user yang tidak kasih rating sama sekali

Number of unique resto:  18
Number of unique users:  136


Normalize the ratings.

In [40]:
scaler = MinMaxScaler()
combined['rating'] = combined['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['rating'].values.reshape(-1,1)))
combined['rating'] = rating_scaled

Abd build the user resto matrix.

In [41]:
combined = combined.drop_duplicates(['userID', 'restoID'])
user_resto_matrix = combined.pivot(index='userID', columns='restoID', values='rating')
user_resto_matrix.fillna(0, inplace=True)

users = user_resto_matrix.index.tolist()
resto = user_resto_matrix.columns.tolist()

user_resto_matrix = user_resto_matrix.values

tf.placeholder only available in v1, so we have to work around. 

In [42]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

We will initialize the TensorFlow placeholder. Then, weights and biases are randomly initialized, the following code are taken from the book: Python Machine Learning Cook Book - Second Edition

In [43]:
num_input = combined['restoID'].nunique()
num_hidden_1 = 64
num_hidden_2 = 32

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

Now, we can build the encoder and decoder model, as follows:

In [44]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

We will construct the model and the predictions

In [45]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

define loss function and optimizer, and minimize the squared error, and define the evaluation metrics

In [46]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Initialize the variables. Because TensorFlow uses computational graphs for its operations, placeholders and variables must be initialized.

In [59]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

We can finally start to train our model.

We split training data into batches, and we feed the network with them.

We train our model with vectors of user ratings, each vector represents a user and each column a resto, and entries are ratings that the user gave to restaurants. 

After a few trials, I discovered that training model for 5 epochs with a batch size of 10 would be consum enough memory. This means that the entire training set will feed our neural network 20 times, every time using 50 users.

In [60]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_resto_matrix.shape[0] / batch_size)
    user_resto_matrix = np.array_split(user_resto_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_resto_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_resto_matrix = np.concatenate(user_resto_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_resto_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['userID', 'restoID', 'rating']
    pred_data['userID'] = pred_data['userID'].map(lambda value: users[value])
    pred_data['restoID'] = pred_data['restoID'].map(lambda value: resto[value])
    
    keys = ['userID', 'restoID']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['userID', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('userID').head(10)

epoch: 1 Loss: 0.45712654789288837
epoch: 2 Loss: 0.4567454556624095
epoch: 3 Loss: 0.45629168550173443
epoch: 4 Loss: 0.45574966073036194
epoch: 5 Loss: 0.45509950319925946
epoch: 6 Loss: 0.4543157716592153
epoch: 7 Loss: 0.4533653159936269
epoch: 8 Loss: 0.45220447580019635
epoch: 9 Loss: 0.45077510674794513
epoch: 10 Loss: 0.4489987889925639
epoch: 11 Loss: 0.4467700719833374
epoch: 12 Loss: 0.4439481794834137
epoch: 13 Loss: 0.4403519034385681
epoch: 14 Loss: 0.4357661008834839
epoch: 15 Loss: 0.42999061942100525
epoch: 16 Loss: 0.4230203926563263
epoch: 17 Loss: 0.4154617687066396
epoch: 18 Loss: 0.408722182114919
epoch: 19 Loss: 0.40393756826718646
epoch: 20 Loss: 0.4009222090244293
epoch: 21 Loss: 0.39888641238212585
epoch: 22 Loss: 0.39725351333618164
epoch: 23 Loss: 0.39572441577911377
epoch: 24 Loss: 0.3941441277662913
epoch: 25 Loss: 0.3924129505952199
epoch: 26 Loss: 0.3904186288515727
epoch: 27 Loss: 0.38792450229326886
epoch: 28 Loss: 0.38419384757677716
epoch: 29 Loss: 0

C:\Users\VINCEN~1\AppData\Local\Temp/ipykernel_23576/1964004144.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_data = pred_data.append(pd.DataFrame(preds))


In [51]:
top_ten_ranked.loc[top_ten_ranked['userID'] == "U1025"]

,userID,restoID,rating
434,U1025,R0003,4.545806e-03
435,U1025,R0004,2.766503e-04
436,U1025,R0005,8.869283e-04
438,U1025,R0007,6.138554e-07
439,U1025,R0008,4.981442e-04
444,U1025,R0013,1.967209e-02
445,U1025,R0014,3.397539e-06
447,U1025,R0016,8.804889e-04
448,U1025,R0017,2.638107e-04


In [53]:
resto_rating.loc[resto_rating['userID'] == "U1025"].sort_values(by=['rating'], ascending=False)

,userID,restoID,rating
16,U1025,R0001,5
53,U1025,R0002,5
347,U1025,R0009,5
397,U1025,R0010,5
447,U1025,R0011,5
640,U1025,R0015,5
197,U1025,R0006,4
497,U1025,R0012,4
760,U1025,R0018,4
